### Note: This file is basically same as 'Candlestick_CombinewithOtherIndicators_Historical15min', but some codes are modified to work with the live Binance API instead.

## To Dos:

-  1. Test Each Combine Strategies and Adjust/Add Indicators, can delete candlestick/indicators based on your own analysis -> To make Accuracy Rate over 55% (Prof Said)
- 2. Make Signal Prediction -> Strong Bullish/Bullish/Weak Bullish + Strong Bearish/Bearish/Weak Bearish (Currently the Live code only include Bullish/Neutal/Bearish and most signals are neutral cuz the combination signal only detect a few bearish/bullish signals)
- 3. Add Evaluation Metrics (Ex. Accuracy Rate, Pnl) for Live API Signal (Currently havent add the performance metrics for Live yet)



In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import talib
import os
import numpy as np
from scipy.signal import dfreqresp


In [2]:
from binance.client import Client
# sys.path.append(os.path.abspath(".."))  # root /PycharmProjects/MMAT
from config.load_env import load_keys

keys = load_keys()
#print("Loaded keys:", keys)
client = Client(keys['api_key'], keys['secret_key'])

###  Connect to Live Binance API:

In [3]:
# sys.path.append(os.path.abspath(".."))  # root /PycharmProjects/MMAT
from config.load_env import load_keys

keys = load_keys()
#print("Loaded keys:", keys)
client = Client(keys['api_key'], keys['secret_key'])

In [4]:
import talib
import os
import time
from binance.client import Client
from dotenv import load_dotenv
try:
    from config.load_env import load_keys
except ImportError:
    # Fallback if config.load_env is unavailable
    def load_keys():
        load_dotenv()
        api_key = os.getenv('BINANCE_API_KEY')
        api_secret = os.getenv('BINANCE_SECRET_KEY')
        if api_key and api_secret:
            return {'api_key': api_key, 'secret_key': api_secret}
        raise ValueError("No API keys found. Set BINANCE_API_KEY and BINANCE_SECRET_KEY in environment or .env file.")

def load_data(csv_path):
    try:
        df = pd.read_csv(csv_path, index_col='timestamp', parse_dates=True)
        df = df[['open', 'high', 'low', 'close', 'volume']].copy()
        print(f"Total K-lines loaded: {len(df)}")
        print("First 5 rows:")
        print(df.head())
        return df
    except FileNotFoundError:
        print(f"CSV file '{csv_path}' not found.")
        return None

def fetch_binance_data(api_key, api_secret, symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_15MINUTE, limit=1000):

    try:
        client = Client(api_key, api_secret)
        klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'num_trades',
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        print(f"Fetched {len(df)} K-lines from Binance API:")
        print(df.head())
        return df
    except Exception as e:
        print(f"Error fetching Binance data: {e}")
        return None

### Resample  to 15 min (Can try other different Time horizon):

In [5]:
def resample_to_15min(df):
    df_15min = df.resample('15min').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()
    print(f"Resampled to 15min, total K-lines: {len(df_15min)}")
    return df_15min

### Select Candlestick Patterns for futher combinations with other Indicators 

#### (Below are the 15 ones that I selected based on the accuracy rate >33% from testing all Candlestick Patterns based on historical 15min, and grouped into Bullish & Bearish Patterns) :

In [6]:
def calculate_patterns(df):
    """
    Detect selected high-accuracy candlestick patterns for further analysis.

    Selection Criteria:
    - Based on historical 15-minute BTC K-line backtest
    - Only include patterns with accuracy > 33%

    Returns:
    - df with added pattern columns
    - patterns dictionary for reference
    """
    patterns = {
        #  Bullish Patterns
        'Hammer': talib.CDLHAMMER,
        'InvertedHammer': talib.CDLINVERTEDHAMMER,
        'BullishEngulfing': lambda o, h, l, c: np.where(talib.CDLENGULFING(o, h, l, c) == 100, 100, 0),
        'PiercingLine': talib.CDLPIERCING,
        'MorningStar': talib.CDLMORNINGSTAR,
        'DragonflyDoji': talib.CDLDRAGONFLYDOJI,
        'LongLine': talib.CDLLONGLINE,

        #  Bearish Patterns
        'HangingMan': talib.CDLHANGINGMAN,
        'ShootingStar': talib.CDLSHOOTINGSTAR,
        'BearishEngulfing': lambda o, h, l, c: np.where(talib.CDLENGULFING(o, h, l, c) == -100, -100, 0),
        'DarkCloudCover': talib.CDLDARKCLOUDCOVER,
        'EveningDojiStar': talib.CDLEVENINGDOJISTAR,
        'EveningStar': talib.CDLEVENINGSTAR,
        'GravestoneDoji': talib.CDLGRAVESTONEDOJI,
        'ThreeLineStrike': talib.CDL3LINESTRIKE,
    }

    # Apply pattern detection and store results in df
    for name, func in patterns.items():
        df[name] = func(df['open'].values, df['high'].values, df['low'].values, df['close'].values)

    
    for name in patterns.keys():
        count = (df[name].abs() > 0).sum()
        print(f"{name} detected {count} times.")


    bullish_patterns = ['Hammer', 'InvertedHammer', 'BullishEngulfing', 'PiercingLine',
                        'MorningStar', 'DragonflyDoji', 'LongLine', 'ThreeLineStrike']

    bearish_patterns = ['HangingMan', 'ShootingStar', 'BearishEngulfing', 'DarkCloudCover',
                        'EveningDojiStar', 'EveningStar', 'GravestoneDoji']

    return df, patterns, bullish_patterns, bearish_patterns


In [7]:
# Ensure we only use past data for signal generation
# This function assumes all indicator values are based on past prices only

def generate_signals(df, patterns, window=1):
    """
    Generate candlestick pattern signals using only current and historical data (no future info).
    Includes special handling for patterns like GravestoneDoji.
    """

    # Initialize signal and direction columns
    for name in patterns.keys():
        df[f'Signal_{name}'] = 0
        df[f'Direction_{name}'] = 'NONE'

    # Grouped by typical TA-Lib output behavior
    bullish_patterns_strong = ['BullishEngulfing', 'ThreeLineStrike']
    bullish_patterns = ['Hammer', 'InvertedHammer', 'PiercingLine',
                        'MorningStar', 'DragonflyDoji', 'LongLine']
    bearish_patterns_strong = ['BearishEngulfing']
    bearish_patterns = ['HangingMan', 'ShootingStar', 'DarkCloudCover', 'EveningDojiStar',
                        'EveningStar']  # GravestoneDoji excluded here due to its output behavior

    for i in range(1, len(df) - window):
        for name in patterns.keys():
            value = df[name].iloc[i]

            # Strong bullish
            if name in bullish_patterns_strong and value == 100:
                df.loc[df.index[i], f'Signal_{name}'] = 1
                df.loc[df.index[i], f'Direction_{name}'] = 'UP'

            # General bullish
            elif name in bullish_patterns and value > 0:
                df.loc[df.index[i], f'Signal_{name}'] = 1
                df.loc[df.index[i], f'Direction_{name}'] = 'UP'

            # Strong bearish
            elif name in bearish_patterns_strong and value == -100:
                df.loc[df.index[i], f'Signal_{name}'] = -1
                df.loc[df.index[i], f'Direction_{name}'] = 'DOWN'

            # General bearish
            elif name in bearish_patterns and value < 0:
                df.loc[df.index[i], f'Signal_{name}'] = -1
                df.loc[df.index[i], f'Direction_{name}'] = 'DOWN'

            # Special case: GravestoneDoji outputs 100 but is bearish
            elif name == 'GravestoneDoji' and value == 100:
                df.loc[df.index[i], f'Signal_{name}'] = -1
                df.loc[df.index[i], f'Direction_{name}'] = 'DOWN'

    return df




### Import Indicators:
#### (Below are RSI, MA5days, MA20days, Volume, ATR, Mean ATR, MACD --> feel free to adjust the time period or add other indicators):

### Current Candlestick Pattern + Technical Indicator Trigger Logic (Can Adjust the conditions):
A combined signal is only considered valid when a specific candlestick pattern appears and at least one of the following technical indicator conditions is met: MA, RSI, MACD, or ATR.

| Category          | Candlestick Patterns                                                                 | Technical Indicator Conditions (at least one must be satisfied)              |
|------------------|----------------------------------------------------------------------------------------|--------------------------------------------------------------------------------|
| Bullish (Strong)  | BullishEngulfing, ThreeLineStrike                                                      | MA5 > MA20, RSI > 50, MACD > MACD_Signal, ATR > mean_ATR, High Volume         |
| Bullish (Moderate)| Hammer, InvertedHammer, PiercingLine, MorningStar, DragonflyDoji, LongLine            | Same as above                                                                 |
| Bearish (Strong)  | BearishEngulfing                                                                      | MA5 < MA20, RSI < 45, MACD < MACD_Signal, ATR > mean_ATR, High Volume         |
| Bearish (Moderate)| HangingMan, ShootingStar, DarkCloudCover, EveningDojiStar, EveningStar, GravestoneDoji| Same as above                                                                 |



In [8]:
# Based on top candlestick patterns + indicator to generate signal
import numpy as np
import talib

# Add High Volume Indicator
def is_high_volume(df, i, lookback=20, threshold=1.5):
    if i < lookback:
        return False
    avg_vol = df['volume'].iloc[i - lookback:i].mean()
    return df['volume'].iloc[i] > threshold * avg_vol

# top candlestick pattern + indicator to generate signal
def generate_combined_signals(df, patterns):
    # Pattern 
    bullish_patterns_strong = ['BullishEngulfing', 'ThreeLineStrike']
    bullish_patterns = ['Hammer', 'InvertedHammer', 'PiercingLine',
                        'MorningStar', 'DragonflyDoji', 'LongLine']
    bearish_patterns_strong = ['BearishEngulfing']
    bearish_patterns = ['HangingMan', 'ShootingStar', 'DarkCloudCover',
                        'EveningDojiStar', 'EveningStar', 'GravestoneDoji']

    # initial
    df['bullish_combined'] = 0
    df['bearish_combined'] = 0
    df['bullish_direction'] = 'NONE'
    df['bearish_direction'] = 'NONE'
    df['bullish_trigger'] = ''
    df['bearish_trigger'] = ''

    # Get Indicators -> Can adjust 
    df['MA5'] = talib.SMA(df['close'], timeperiod=5)
    df['MA20'] = talib.SMA(df['close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['mean_ATR'] = df['ATR'].rolling(window=20).mean()
    df['MACD'], df['MACD_Signal'], _ = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)

    # Signal columns
    for name in patterns.keys():
        if name == 'GravestoneDoji':
            df[f'Signal_{name}'] = df[name].apply(lambda x: -1 if x == 100 else 0)
        elif name in bullish_patterns_strong:
            df[f'Signal_{name}'] = df[name].apply(lambda x: 1 if x == 100 else 0)
        elif name in bearish_patterns_strong:
            df[f'Signal_{name}'] = df[name].apply(lambda x: -1 if x == -100 else 0)
        else:
            df[f'Signal_{name}'] = df[name].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)

    # Signal Trigger Condition
    for i in range(1, len(df) - 1):
        volume_condition = is_high_volume(df, i)
        ma_condition_bull = df['MA5'].iloc[i] > df['MA20'].iloc[i]
        ma_condition_bear = df['MA5'].iloc[i] < df['MA20'].iloc[i]
        rsi_condition_bull = df['RSI'].iloc[i] > 50
        rsi_condition_bear = df['RSI'].iloc[i] < 45
        macd_bull = df['MACD'].iloc[i] > df['MACD_Signal'].iloc[i]
        macd_bear = df['MACD'].iloc[i] < df['MACD_Signal'].iloc[i]
        atr_condition = df['ATR'].iloc[i] > df['mean_ATR'].iloc[i]

        # Bullish pattern
        for pattern in bullish_patterns + bullish_patterns_strong:
            if df[f'Signal_{pattern}'].iloc[i] == 1:
                trigger_parts = [pattern]
                if ma_condition_bull:
                    trigger_parts.append('MA5>MA20')
                if rsi_condition_bull:
                    trigger_parts.append('RSI>50')
                if macd_bull:
                    trigger_parts.append('MACD_Bullish')
                if atr_condition:
                    trigger_parts.append('HighVolatility')
                if volume_condition:
                    trigger_parts.append('HighVolume')
                if len(trigger_parts) >= 2:
                    df.loc[df.index[i], 'bullish_combined'] = 1
                    df.loc[df.index[i], 'bullish_direction'] = 'UP'
                    df.loc[df.index[i], 'bullish_trigger'] = ' + '.join(trigger_parts)

        # Bearish pattern
        for pattern in bearish_patterns + bearish_patterns_strong:
            if df[f'Signal_{pattern}'].iloc[i] == -1:
                trigger_parts = [pattern]
                if ma_condition_bear:
                    trigger_parts.append('MA5<MA20')
                if rsi_condition_bear:
                    trigger_parts.append('RSI<45')
                if macd_bear:
                    trigger_parts.append('MACD_Bearish')
                if atr_condition:
                    trigger_parts.append('HighVolatility')
                if volume_condition:
                    trigger_parts.append('HighVolume')
                if len(trigger_parts) >= 2:
                    df.loc[df.index[i], 'bearish_combined'] = -1
                    df.loc[df.index[i], 'bearish_direction'] = 'DOWN'
                    df.loc[df.index[i], 'bearish_trigger'] = ' + '.join(trigger_parts)
       
    print("Bullish Signal Count:", df['bullish_combined'].sum())
    print("Bearish Signal Count:", -df['bearish_combined'].sum())
    print(df[df['bullish_combined'] == 1][['close', 'bullish_trigger']].tail(5))
    print(df[df['bearish_combined'] == -1][['close', 'bearish_trigger']].tail(5))


    return df


### Evaluation: (Currently use Accuracy rates)

In [9]:
def evaluate_patterns(df, patterns, window=1, threshold=0.0005):
    """
    Evaluate the accuracy of each candlestick pattern signal.

    This function measures by calculating the forward return after each signal and comparing it against a defined threshold.

    Parameters:
    
    (1) window :  default=1
        Holding period in bars/candles to compute future returns (e.g., 1 bar ahead).

    (2) threshold : default=0.0005
        Minimum return required for a signal to be considered successful (e.g., 0.05%).

    """
    results = {}

    #  Compute future return
    df['next_close'] = df['close'].shift(-window)
    df['return'] = (df['next_close'] - df['close']) / df['close']

    # Evaluate each raw candlestick pattern signal
    for name in list(patterns.keys()):
        signal_col = f'Signal_{name}'
        signals = df[df[signal_col] != 0]
        total_signals = len(signals)

        if total_signals == 0:
            results[name] = {'accuracy': 0, 'total_signals': 0, 'correct_signals': 0}
            continue

        correct_signals = len(signals[
            ((signals[signal_col] == 1) & (df.loc[signals.index, 'return'] >= threshold)) |
            ((signals[signal_col] == -1) & (df.loc[signals.index, 'return'] <= -threshold))
        ])
        accuracy = correct_signals / total_signals * 100

        results[name] = {
            'accuracy': accuracy,
            'total_signals': total_signals,
            'correct_signals': correct_signals
        }

    #  Evaluate combined signals (pattern + indicator)
    for signal_col in ['bullish_combined', 'bearish_combined']:
        signals = df[df[signal_col] != 0]
        total_signals = len(signals)

        if total_signals == 0:
            results[signal_col] = {'accuracy': 0, 'total_signals': 0, 'correct_signals': 0}
            continue

        correct_signals = len(signals[
            ((signals[signal_col] == 1) & (df.loc[signals.index, 'return'] >= threshold)) |
            ((signals[signal_col] == -1) & (df.loc[signals.index, 'return'] <= -threshold))
        ])
        accuracy = correct_signals / total_signals * 100

        results[signal_col] = {
            'accuracy': accuracy,
            'total_signals': total_signals,
            'correct_signals': correct_signals
        }

    return results


### Plot Realtime result:

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

import plotly.io as pio
pio.renderers.default = "browser"


def plot_realtime_signals(df, symbol='BTCUSDT', data_range=50, output_dir='/Users/wynn/PycharmProjects/MMAT/plots/'):
    """
    Generate a real-time chart for the latest candlesticks and signals.
    """
    # Take the last `data_range` rows for plotting
    df_plot = df.iloc[-data_range:].copy()

    # Get combined signals
    up_signals = df_plot[df_plot['bullish_combined'] == 1]
    down_signals = df_plot[df_plot['bearish_combined'] == -1]
    print(f"Plotting Real-Time: {len(up_signals)} Bullish UP signals, {len(down_signals)} Bearish DOWN signals in last {data_range} K-lines")

    # Create subplots
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=['Candlestick + MA', 'RSI', 'ATR', 'Volume'],
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    # Candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=df_plot.index,
            open=df_plot['open'],
            high=df_plot['high'],
            low=df_plot['low'],
            close=df_plot['close'],
            name='Candlestick',
            increasing_line_color='green',
            decreasing_line_color='red'
        ),
        row=1, col=1
    )

    # Moving Averages
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot['MA5'],
            mode='lines',
            name='5 MA',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot['MA20'],
            mode='lines',
            name='20 MA',
            line=dict(color='purple')
        ),
        row=1, col=1
    )

    # Bullish signals
    fig.add_trace(
        go.Scatter(
            x=up_signals.index,
            y=up_signals['close'] * 1.005,
            mode='markers',
            marker=dict(symbol='triangle-up', color='green', size=10),
            name='Bullish Signal',
            text=up_signals['bullish_trigger'],
            hoverinfo='text+x+y'
        ),
        row=1, col=1
    )

    # Bearish signals
    fig.add_trace(
        go.Scatter(
            x=down_signals.index,
            y=down_signals['close'] * 0.995,
            mode='markers',
            marker=dict(symbol='triangle-down', color='red', size=10),
            name='Bearish Signal',
            text=down_signals['bearish_trigger'],
            hoverinfo='text+x+y'
        ),
        row=1, col=1
    )

    # RSI subplot
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot['RSI'],
            mode='lines',
            name='RSI',
            line=dict(color='blue')
        ),
        row=2, col=1
    )
    fig.add_hline(y=50, line_dash='dash', line_color='black', row=2, col=1)

    # ATR subplot
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot['ATR'],
            mode='lines',
            name='ATR',
            line=dict(color='orange')
        ),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot['mean_ATR'] * 1.2,
            mode='lines',
            name='1.2 * mean_ATR',
            line=dict(color='red', dash='dash')
        ),
        row=3, col=1
    )

    # Volume subplot
    fig.add_trace(
        go.Bar(
            x=df_plot.index,
            y=df_plot['volume'],
            name='Volume',
            marker_color='blue'
        ),
        row=4, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot['Volume_MA20'] * 1.3,
            mode='lines',
            name='1.3 * Volume_MA20',
            line=dict(color='red', dash='dash')
        ),
        row=4, col=1
    )

    # Update layout
    fig.update_layout(
        title=f'May31 Real-Time Signals for {symbol}',
        xaxis_title='Time',
        yaxis_title='Price ($)',
        yaxis2_title='RSI',
        yaxis3_title='ATR',
        yaxis4_title='Volume',
        xaxis_rangeslider_visible=False,
        showlegend=True,
        height=800,
        template='plotly_white'
    )

    # Save and display
    os.makedirs(output_dir, exist_ok=True)
    html_path = os.path.join(output_dir, f'realtime_signals_{symbol}.html')
    try:
        fig.write_html(html_path)
        print(f"Saved real-time plot to {html_path}")
        fig.show(renderer="browser")  # Opens in browser
    except Exception as e:
        print(f"Error displaying real-time plot: {e}")
        fig.write_html(html_path)
        print(f"Saved real-time plot to {html_path} as fallback")



### Run Realtime Data:

In [11]:
def run_realtime_signals(api_key, api_secret, symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_15MINUTE, limit=100, sleep_seconds=900):
    print("Starting real-time signal generation. Press Ctrl+C to stop.")
    while True:
        try:
            df = fetch_binance_data(api_key, api_secret, symbol, interval, limit)
            if df is None:
                print("Failed to fetch data. Retrying in 60 seconds.")
                time.sleep(60)
                continue

            # Calculate technical indicators
            df['MA5'] = talib.SMA(df['close'], timeperiod=5)
            df['MA20'] = talib.SMA(df['close'], timeperiod=20)
            df['RSI'] = talib.RSI(df['close'], timeperiod=14)
            df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
            df['mean_ATR'] = df['ATR'].rolling(window=20).mean()
            df['MACD'], df['MACD_Signal'], _ = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
            df['Volume_MA20'] = talib.SMA(df['volume'], timeperiod=20)

            df, patterns, bullish_patterns, bearish_patterns = calculate_patterns(df)
            df = generate_signals(df, patterns)  # Assumes generate_signals from previous response
            df = generate_combined_signals(df, patterns)


            latest_signal = df.iloc[-1][[f'Signal_{name}' for name in patterns.keys()]]
            timestamp = df.index[-1]

            print(f"\n[{timestamp}] Latest Signals:")
            for name in patterns.keys():
                signal = latest_signal[f'Signal_{name}']
                direction = 'UP' if signal == 1 else 'DOWN' if signal == -1 else 'NONE'
                print(f"{name}: {direction}")

            # Combined signals for prediction
            bullish_signal = df['bullish_combined'].iloc[-1]
            bearish_signal = df['bearish_combined'].iloc[-1]
            print(f"Bullish Combined: {df['bullish_direction'].iloc[-1]}")
            if bullish_signal == 1:
                print(f"Bullish Trigger: {df['bullish_trigger'].iloc[-1]}")
            print(f"Bearish Combined: {df['bearish_direction'].iloc[-1]}")
            if bearish_signal == -1:
                print(f"Bearish Trigger: {df['bearish_trigger'].iloc[-1]}")

            # Predict next 15-minute direction
            if bullish_signal == 1 and bearish_signal != -1:
                print(f"\nPrediction for next 15min: UP")
            elif bearish_signal == -1 and bullish_signal != 1:
                print(f"\nPrediction for next 15min: DOWN")
            else:
                print(f"\nPrediction for next 15min: NEUTRAL (Conflicting or no signals)")

            # Generate real-time plot
            plot_realtime_signals(df, symbol, data_range=50)

            print(f"Waiting {sleep_seconds} seconds for next update...")
            time.sleep(sleep_seconds)

        except KeyboardInterrupt:
            print("\nStopped real-time signal generation.")
            break
        except Exception as e:
            print(f"Error in real-time loop: {e}. Retrying in 60 seconds.")
            time.sleep(60)

### Def Main:

In [ ]:
def main(realtime=False):
    use_api = True
    csv_path = '/Users/wynn/PycharmProjects/MMAT/data/btc_1min.csv' # Adjust to yours

    if use_api:
        try:
            keys = load_keys()
            api_key = keys['api_key']
            api_secret = keys['secret_key']
        except Exception as e:
            print(f"Failed to load keys: {e}")
            print("Ensure config.load_env.load_keys is set up or use environment variables BINANCE_API_KEY and BINANCE_SECRET_KEY.")
            return

        if realtime:
            run_realtime_signals(api_key, api_secret)
            return

        df = fetch_binance_data(api_key, api_secret)
        if df is None:
            print("Failed to fetch Binance data, exiting.")
            return
    else:
        df = load_data(csv_path)
        if df is None:
            return
        df = resample_to_15min(df)

    #  Apply candlestick pattern detection using TA-Lib
    df, patterns, bullish_patterns, bearish_patterns = calculate_patterns(df)

    # Generate basic pattern-only signals (e.g., Signal_Hammer, Signal_Engulfing)
    df = generate_signals(df, patterns)

    # Generate combined signals (pattern + indicator confirmation)
    df = generate_combined_signals(df, patterns)

    #  Evaluate signal accuracy based on forward returns
    accuracy_results = evaluate_patterns(df, patterns)

    # pattern evaluation results
    print("\n--- Pattern Signal Accuracy ---")
    for name, metrics in sorted(accuracy_results.items(), key=lambda x: x[1]['accuracy'], reverse=True):
        print(f"{name} - Accuracy: {metrics['accuracy']:.2f}%, Total: {metrics['total_signals']}, Correct: {metrics['correct_signals']}")

    # Plot signal charts
    #plot_pattern_results(df, patterns, 'BTC')
    plot_combined_results(df, 'BTC')

    plot_pattern_results(df, patterns, 'BTC', max_points=1000, data_range=1000)

if __name__ == "__main__":
    main(realtime=True)

Starting real-time signal generation. Press Ctrl+C to stop.
Fetched 100 K-lines from Binance API:
                          open       high        low      close     volume
timestamp                                                                 
2025-05-30 11:15:00  105815.90  105943.97  105720.56  105773.52  134.84726
2025-05-30 11:30:00  105773.51  105901.99  105757.29  105802.39  106.14119
2025-05-30 11:45:00  105802.39  105963.27  105700.00  105880.51  156.47407
2025-05-30 12:00:00  105880.50  105975.80  105153.84  105342.30  400.41804
2025-05-30 12:15:00  105342.30  105551.70  105255.36  105551.04  249.11120
Hammer detected 1 times.
InvertedHammer detected 0 times.
BullishEngulfing detected 5 times.
PiercingLine detected 0 times.
MorningStar detected 0 times.
DragonflyDoji detected 2 times.
LongLine detected 15 times.
HangingMan detected 2 times.
ShootingStar detected 0 times.
BearishEngulfing detected 3 times.
DarkCloudCover detected 0 times.
EveningDojiStar detected 0 times.
E